In [ ]:
# =========================
# Imports
# =========================
import os
import sys
import glob
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from functools import partial
from scipy.signal import savgol_filter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# For ResNet-style pieces (explicit names used later)
from tensorflow.keras.layers import (
    BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D,
    MaxPooling2D, ZeroPadding2D, Add
)

# =========================
# Classification part
# =========================
path = '/content/drive/MyDrive/Colab Notebooks/cnn_trainset_2_3/*'
files = glob.glob(path)
print("Sample files:", files[:5])

# Train - Val - Test split
train, test = train_test_split(files, test_size=0.2, random_state=10)
train, val = train_test_split(train, test_size=0.2, random_state=10)
print("Counts -> train/val/test:", len(train), len(val), len(test))

number_of_classes = 3
labels = ["n", "o", "r"]  # ['Normal','Overload','Recovery'] abbreviations
encoder = LabelEncoder()
encoder.fit(labels)

# =========================
# Data Generator
# =========================
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, files, encoder, batch_size=32, dim=(256, 256), n_channels=10,
                 n_classes=3, shuffle=True):
        'Initialization'
        self.files = files
        self.dim = dim
        self.batch_size = batch_size
        self.encoder = encoder
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.files) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.files[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def get_data(self):
        # Return all data in one go (memory permitting)
        list_IDs_temp = [self.files[k] for k in self.indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.files))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        X = np.empty((self.batch_size, *self.dim, self.n_channels), dtype=np.float32)
        y = np.empty((self.batch_size), dtype=int)

        for i, ID in enumerate(list_IDs_temp):
            # Each file is assumed to be a saved numpy array with shape (256,256,10)
            X[i,] = np.load(ID)
            # Class label is the first character of the filename base (e.g., 'n***.npy')
            label_char = os.path.basename(ID)[0]
            y[i] = self.encoder.transform([label_char])[0]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

# Generators
training_generator   = DataGenerator(train, encoder=encoder, shuffle=True)
validation_generator = DataGenerator(val,   encoder=encoder, shuffle=True)
test_generator       = DataGenerator(test,  encoder=encoder, batch_size=1, shuffle=False)

# =========================
# VGG16-style model
# =========================
def build_vgg16_like(input_shape=(256, 256, 10), n_classes=3):
    input_tensor = Input(shape=input_shape, dtype='float32', name='input')

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(input_tensor)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(4096, kernel_initializer='he_normal', activation='relu')(x)
    x = layers.Dense(4096, kernel_initializer='he_normal', activation='relu')(x)
    output_tensor = layers.Dense(units=n_classes, activation="softmax")(x)

    model = Model(input_tensor, output_tensor, name="VGG16_like")
    return model

# =========================
# ResNet50-style model (from your blocks)
# =========================
def build_resnet50_like(input_shape=(256, 256, 10), n_classes=3):
    input_tensor = Input(shape=input_shape, dtype='float32', name='input')

    def conv1_layer(x):
        x = ZeroPadding2D(padding=(3, 3))(x)
        x = Conv2D(64, (7, 7), strides=(2, 2))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        return x

    def conv2_layer(x):
        x = MaxPooling2D((3, 3), strides=2)(x)
        shortcut = x
        for i in range(3):
            if i == 0:
                x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
                shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)

                x = BatchNormalization()(x)
                shortcut = BatchNormalization()(shortcut)
                x = Add()([x, shortcut])
                x = Activation('relu')(x)
                shortcut = x
            else:
                x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Add()([x, shortcut])
                x = Activation('relu')(x)
                shortcut = x
        return x

    def conv3_layer(x):
        shortcut = x
        for i in range(4):
            if i == 0:
                x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
                shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)

                x = BatchNormalization()(x)
                shortcut = BatchNormalization()(shortcut)
                x = Add()([x, shortcut])
                x = Activation('relu')(x)
                shortcut = x
            else:
                x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Add()([x, shortcut])
                x = Activation('relu')(x)
                shortcut = x
        return x

    def conv4_layer(x):
        shortcut = x
        for i in range(6):
            if i == 0:
                x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
                shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)

                x = BatchNormalization()(x)
                shortcut = BatchNormalization()(shortcut)
                x = Add()([x, shortcut])
                x = Activation('relu')(x)
                shortcut = x
            else:
                x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Add()([x, shortcut])
                x = Activation('relu')(x)
                shortcut = x
        return x

    def conv5_layer(x):
        shortcut = x
        for i in range(3):
            if i == 0:
                x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
                shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)

                x = BatchNormalization()(x)
                shortcut = BatchNormalization()(shortcut)
                x = Add()([x, shortcut])
                x = Activation('relu')(x)
                shortcut = x
            else:
                x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
                x = BatchNormalization()(x)
                x = Activation('relu')(x)

                x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
                x = BatchNormalization()(x)

                x = Add()([x, shortcut])
                x = Activation('relu')(x)
                shortcut = x
        return x

    x = conv1_layer(input_tensor)
    x = conv2_layer(x)
    x = conv3_layer(x)
    x = conv4_layer(x)
    x = conv5_layer(x)
    x = GlobalAveragePooling2D()(x)
    output_tensor = Dense(units=n_classes, activation='softmax')(x)

    model = Model(input_tensor, output_tensor, name="ResNet50_like")
    return model

# =========================
# Choose model: 'VGG16' or 'ResNet50'
# =========================
MODEL_NAME = 'ResNet50'  # change to 'VGG16' if desired

if MODEL_NAME == 'VGG16':
    model = build_vgg16_like(input_shape=(256, 256, 10), n_classes=number_of_classes)
else:
    model = build_resnet50_like(input_shape=(256, 256, 10), n_classes=number_of_classes)

model.summary()

# =========================
# Compile & Train (simple defaults; adjust as needed)
# =========================
model.compile(optimizer=keras.optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

ckpt_path = f'/content/drive/MyDrive/Colab Notebooks/{MODEL_NAME}_best.h5'
callbacks = [
    ModelCheckpoint(ckpt_path, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_accuracy', mode='max', patience=8, restore_best_weights=True)
]

history = model.fit(
    training_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=callbacks,
    verbose=1
)

# =========================
# Evaluate & Confusion Matrix
# =========================
# Ground-truth labels for test set from filenames
y_true_chars = [os.path.basename(p)[0] for p in test]
y_true = encoder.transform(y_true_chars)

# Predictions
y_pred_probs = model.predict(test_generator, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)

# Build confusion matrix in the same label order you defined
inv_labels = labels  # ["n","o","r"]
matrix = confusion_matrix(y_true, y_pred, labels=encoder.transform(inv_labels))

# As a DataFrame with row/col labels matching your snippet
dataframe = pd.DataFrame(matrix, index=inv_labels, columns=inv_labels)

# Plot
plt.figure(figsize=(8, 8))
label_font = {'size': 12}
sns.heatmap(dataframe, annot=True, cmap="Blues", linewidths=1, square=True,
            cbar=False, fmt='g')
plt.yticks(rotation=0)
plt.title("Confusion Matrix_ResNet50" if MODEL_NAME == 'ResNet50' else "Confusion Matrix_VGG16")
plt.ylabel('Predicted labels', fontdict=label_font)
plt.xlabel('True labels', fontdict=label_font)

# Save exactly to your provided path (kept unchanged)
plt.savefig(r'/content/drive/MyDrive/Colab Notebooks/ALL_ResNet50.jpg', bbox_inches='tight')
plt.close()

print("Saved confusion matrix figure to: /content/drive/MyDrive/Colab Notebooks/ALL_ResNet50.jpg")